In [0]:
# Fetch parameters from Azure Data Factory
#schemaName=dbutils.widgets.get("schemaName")
#tableName=dbutils.widgets.get("tableName")
#filePath=dbutils.widgets.get("filePath")

schemaName="adventureworks"
tableName="Address"
filePath="20241022"

# Fetch schema data from landing zone
jsonSchema = spark.read.parquet(f"/Volumes/buildingma/volumes/mnt_landing/\
{schemaName}/{filePath}/{tableName}.parquet").schema.json()
ddl = spark.sparkContext._jvm.org.apache.spark.sql.types.DataType \
  .fromJson(jsonSchema).toDDL()

# Migrate parquet data to delta files
(spark.readStream
  .format("cloudFiles")
  .option("cloudFiles.format", "parquet")
  .option("cloudFiles.includeExistingFiles", "true")
  .option("cloudFiles.backfillInterval", "1 day")
  .option("cloudFiles.schemaLocation", f"/Volumes/buildingma/volumes/mnt_landing/\
  {schemaName}/_checkpoint/{tableName}_autoload/")
  .schema(ddl)
  .load(f"/Volumes/buildingma/volumes/mnt_landing/\
  {schemaName}/{filePath}/{tableName}.parquet")
  .writeStream
  .format("delta")
  .option("checkpointLocation", f"/Volumes/buildingma/volumes/mnt_landing/\
  {schemaName}/_checkpoint/{tableName}_autoload/")
  .trigger(availableNow=True)
  .toTable(f"bronze_adventureworks.{tableName}")
)

---------------------------------------------------------------------------
PySparkAttributeError                     Traceback (most recent call last)
File <command-3511281742453697>, line 13
     10 # Fetch schema data from landing zone
     11 jsonSchema = spark.read.parquet(f"/Volumes/buildingma/volumes/mnt_landing/\
     12 {schemaName}/{filePath}/{tableName}.parquet").schema.json()
---> 13 ddl = spark.sparkContext._jvm.org.apache.spark.sql.types.DataType \
     14   .fromJson(jsonSchema).toDDL()
     16 # Migrate parquet data to delta files
     17 (spark.readStream
     18   .format("cloudFiles")
     19   .option("cloudFiles.format", "parquet")
   (...)
     32   .toTable(f"bronze_adventureworks.{tableName}")
     33 )

File /databricks/spark/python/pyspark/sql/connect/session.py:967, in SparkSession.__getattr__(self, name)
    965 def __getattr__(self, name: str) -> Any:
    966     if name in ["_jsc", "_jconf", "_jvm", "_jsparkSession", "sparkContext", "newSession"]:
--> 967 